 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Prepare-the-data" data-toc-modified-id="Prepare-the-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prepare the data</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Theano-Implementation" data-toc-modified-id="Theano-Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Theano Implementation</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Train-for-a-single-day" data-toc-modified-id="Train-for-a-single-day-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Train for a single day</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Post-processing-for-the-entire-period" data-toc-modified-id="Post-processing-for-the-entire-period-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Post processing for the entire period</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Post-processing-for-2016---benchmark-for-later-models" data-toc-modified-id="Post-processing-for-2016---benchmark-for-later-models-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Post-processing for 2016 - benchmark for later models</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Keras-implementation" data-toc-modified-id="Keras-implementation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Keras implementation</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Predict-for-one-day" data-toc-modified-id="Predict-for-one-day-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Predict for one day</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Post-processing-for-2016" data-toc-modified-id="Post-processing-for-2016-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Post-processing for 2016</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Train-2015,-predict-2016" data-toc-modified-id="Train-2015,-predict-2016-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Train 2015, predict 2016</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Fully-connected-model" data-toc-modified-id="Fully-connected-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Fully connected model</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#For-one-day" data-toc-modified-id="For-one-day-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>For one day</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Train-for-the-entire-period" data-toc-modified-id="Train-for-the-entire-period-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Train for the entire period</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#With-hidden-layers" data-toc-modified-id="With-hidden-layers-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>With hidden layers</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Get-auxiliary-data" data-toc-modified-id="Get-auxiliary-data-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Get auxiliary data</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Check-out-dataset" data-toc-modified-id="Check-out-dataset-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Check out dataset</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Data-loading-and-preparing-functions" data-toc-modified-id="Data-loading-and-preparing-functions-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Data loading and preparing functions</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Test-with-hidden-layer-model" data-toc-modified-id="Test-with-hidden-layer-model-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Test with hidden layer model</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Try-out-an-embedding" data-toc-modified-id="Try-out-an-embedding-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Try out an embedding</a></span></li></ul></li></ul></div>

# EMOS Network

The goal of this notebooks is to build and test a network implementation of EMOS, once in pure theano and then in keras. First we will try to replicate the results of the standard global EMOS. 

The reference period is always the mean CRPS for all dates and stations in 2016.

In [50]:
# Imports
# Note that the cost function only works with the theano backend for keras
from importlib import reload
import EMOS_network_theano; reload(EMOS_network_theano)
from  EMOS_network_theano import EMOS_Network
from crps_loss import crps_cost_function
import utils; reload(utils)
from utils import *
from datetime import datetime
%matplotlib inline
import matplotlib.pyplot as plt

In [9]:
# Basic setup
DATA_DIR = '/Volumes/STICK/data/ppnn_data/'  # Mac
# DATA_DIR = '/project/meteo/w2w/C7/ppnn_data/'   # LMU
fn = 'data_interpolated_00UTC.nc'   # Temperature observation and forecast data
window_size = 25   # Days in rolling window
fclt = 48   # Forecast lead time in hours

## Prepare the data

The interpolated data set contains the observations and forecasts from the 50 members for each station and each day.

In [7]:
# Load the full dataset
tobs_full, tfc_full, dates = load_nc_data(DATA_DIR + fn)
tobs_full.shape, tfc_full.shape, dates.shape

((3653, 537), (3653, 50, 537), (3653,))

Now we convert this dataset to use as the input for the EMOS networks. For this we pick a forecast date, and return as the training data all previous days within the window size previous to the start of the forecast. The test data is simply the data for the chosen forecast date. The 50 member ensemble is summarized by the mean and the standard deviation. Additionally, we remove all data where the observations are missing. Finally, the inputs are scaled. For this we subtract the mean from the mean and divide both the mean and the standard deviation by their standard deviations.

In [17]:
# Let's chose a random day to illustrate this
date_idx = return_date_idx(dates, 2011, 2, 14)   # year, month, day
date_idx

1503

In [18]:
# Now get the training and test data for this day
tfc_mean_train, tfc_std_train, tobs_train, tfc_mean_test, tfc_std_test, tobs_test = \
        get_train_test_data(tobs_full, tfc_full, date_idx, window_size, fclt, 
                            subtract_std_mean=False)

In [19]:
tfc_mean_train.shape, tfc_mean_test.shape

((12619,), (503,))

These 1D arrays contain the data for all dates and stations.

## Theano Implementation

To start with we build the model in pure theano. The model is defined in a separate script `EMOS_network_theano.py`. The network uses a custom CRPS loss function which is defined in `crps_loss.py`.

The EMOS_Network class is build to work in a similar way to keras models. For the fitting we are using gradient descent. Since we are using the entire dataset for each update, it is not stochastic. An early stopping algorithm is built into the fitting function. It stops training if the average training CRPS of the last 5 steps is decreasing by less than a parameter delta. 

### Train for a single day

To illustrate how the model work we will use the data for our example day above and fit the model.

In [24]:
# Define some model parameters
lr = np.asarray(0.1, dtype='float32')   # The learning rate
early_stopping_delta = 1e-4   # How much the CRPS must improve before stopping
steps_max = 1000   # How many steps to fit at max

In [25]:
# Set up the theano model
model_theano = EMOS_Network()

In [26]:
# Train the model for some steps
model_theano.fit(tfc_mean_train, tfc_std_train, tobs_train, steps_max, 
          (tfc_mean_test, tfc_std_test, tobs_test), lr=lr, 
          early_stopping_delta=early_stopping_delta)
# Output is the training CRPS and the test CRPS

(array(1.1293686004034018), array(0.7828335055061354))

### Post processing for the entire period

To compare the network model with the standard EMOS we will run it from 1 January 2008 to 31 December 2016. When looping over the days we are not resetting the model weights for each day. This drastically reduces the training time with identical results.

In [38]:
# Get start and stop indices
date_idx_start = return_date_idx(dates, 2008, 1, 1)
date_idx_stop = return_date_idx(dates, 2016, 12, 31)
date_idx_start, date_idx_stop

(363, 3650)

In [39]:
model_theano = EMOS_Network()

In [40]:
# This function loops over the days.
train_crps_list, valid_crps_list = loop_over_days(
    model_theano,
    tobs_full, 
    tfc_full, 
    date_idx_start, date_idx_stop, 
    window_size=window_size,
    fclt=fclt,     
    epochs_max=steps_max, 
    early_stopping_delta=early_stopping_delta, 
    lr=lr,
    model_type='EMOS_Network_theano')

400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
Time: 200.71 s


In [41]:
# Let's see what the mean prediction CRPS is
np.mean(valid_crps_list)

1.055801828538059

In [42]:
np.save('./results/EMOS_network_theano.npy', valid_crps_list)

The standard EMOS global score (in `standard_postprocessing/emos_global.R`) is 1.0654. So we are doing a little better even, but this might just be chance.

### Post-processing for 2016 - benchmark for later models

Now the same process but only for the year of 2016. We can use this as a benchmark for later models.

In [43]:
date_idx_start = return_date_idx(dates, 2016, 1, 1)
date_idx_stop = return_date_idx(dates, 2016, 12, 31)
date_idx_start, date_idx_stop

(3285, 3650)

In [44]:
train_crps_list, valid_crps_list = loop_over_days(
    model_theano,
    tobs_full, 
    tfc_full, 
    date_idx_start, date_idx_stop, 
    window_size=window_size,
    fclt=fclt,     
    epochs_max=steps_max, 
    early_stopping_delta=early_stopping_delta, 
    lr=lr,
    model_type='EMOS_Network_theano')

3300
3400
3500
3600
Time: 24.08 s


In [45]:
np.mean(train_crps_list), np.mean(valid_crps_list)

(0.98131095989952555, 0.99854569302246976)

So the global EMOS benchmark for 2016 is around 1.00. 

## Keras implementation

Now let's build the same model in keras. This will provide a good starting point to expand the model later on.

In [62]:
# import the keras modules
# Note that the cost function only works with the theano backend
import keras
from keras.layers import Input, Dense, merge, Embedding, Flatten, Dropout
from keras.models import Model
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam

In [47]:
# Let's build the model with Keras' functional API
# This is quite a bit easier and shorter than in theano 
def build_EMOS_Network_keras():
    mean_in = Input(shape=(1,))
    std_in = Input(shape=(1,))
    mean_out = Dense(1, activation='linear')(mean_in)
    std_out = Dense(1, activation='linear')(std_in)
    x = keras.layers.concatenate([mean_out, std_out], axis=1)
    return Model(inputs=[mean_in, std_in], outputs=x)

In [48]:
model_keras = build_EMOS_Network_keras()

In [49]:
model_keras.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 1)             2           input_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 1)             2           input_2[0][0]                    
___________________________________________________________________________________________

In [52]:
# Compile the model with SGD optimizer and our custom loss function
opt = SGD(lr=0.1)  
model_keras.compile(optimizer=opt, loss=crps_cost_function, 
                    metrics=[crps_cost_function])

### Predict for one day

In keras the early stopping algorithm works slightly differently. It stops training once the training loss hasn't decreased by an amount delta in a certain number of steps (patience).

In [53]:
# This way we have the gradient descent on the whole training set just as in theano
batch_size = tfc_mean_train.shape[0]   

In [54]:
h = model_keras.fit([tfc_mean_train, tfc_std_train], tobs_train, epochs=steps_max, batch_size=batch_size,
          validation_data=[[tfc_mean_test, tfc_std_test], tobs_test], verbose=1,
          callbacks=[EarlyStopping(monitor='loss', min_delta=early_stopping_delta,
                                  patience=2)])

Train on 12619 samples, validate on 503 samples
Epoch 1/1000
12619/12619 [==============================] - 0s - loss: 3.3206 - crps_cost_function: 3.3206 - val_loss: 1.6199 - val_crps_cost_function: 1.6199
Epoch 2/1000
12619/12619 [==============================] - 0s - loss: 3.2698 - crps_cost_function: 3.2698 - val_loss: 1.6149 - val_crps_cost_function: 1.6149
Epoch 3/1000
12619/12619 [==============================] - 0s - loss: 3.2210 - crps_cost_function: 3.2210 - val_loss: 1.6103 - val_crps_cost_function: 1.6103
Epoch 4/1000
12619/12619 [==============================] - 0s - loss: 3.1739 - crps_cost_function: 3.1739 - val_loss: 1.6058 - val_crps_cost_function: 1.6058
Epoch 5/1000
12619/12619 [==============================] - 0s - loss: 3.1285 - crps_cost_function: 3.1285 - val_loss: 1.6015 - val_crps_cost_function: 1.6015
Epoch 6/1000
12619/12619 [==============================] - 0s - loss: 3.0847 - crps_cost_function: 3.0847 - val_loss: 1.5973 - val_crps_cost_function: 1.597

12619/12619 [==============================] - 0s - loss: 1.8254 - crps_cost_function: 1.8254 - val_loss: 1.2736 - val_crps_cost_function: 1.2736
Epoch 52/1000
12619/12619 [==============================] - 0s - loss: 1.8062 - crps_cost_function: 1.8062 - val_loss: 1.2641 - val_crps_cost_function: 1.2641
Epoch 53/1000
12619/12619 [==============================] - 0s - loss: 1.7874 - crps_cost_function: 1.7874 - val_loss: 1.2545 - val_crps_cost_function: 1.2545
Epoch 54/1000
12619/12619 [==============================] - 0s - loss: 1.7688 - crps_cost_function: 1.7688 - val_loss: 1.2449 - val_crps_cost_function: 1.2449
Epoch 55/1000
12619/12619 [==============================] - 0s - loss: 1.7506 - crps_cost_function: 1.7506 - val_loss: 1.2353 - val_crps_cost_function: 1.2353
Epoch 56/1000
12619/12619 [==============================] - 0s - loss: 1.7326 - crps_cost_function: 1.7326 - val_loss: 1.2257 - val_crps_cost_function: 1.2257
Epoch 57/1000
12619/12619 [===========================

12619/12619 [==============================] - 0s - loss: 1.2301 - crps_cost_function: 1.2301 - val_loss: 0.8882 - val_crps_cost_function: 0.8882
Epoch 103/1000
12619/12619 [==============================] - 0s - loss: 1.2256 - crps_cost_function: 1.2256 - val_loss: 0.8844 - val_crps_cost_function: 0.8844
Epoch 104/1000
12619/12619 [==============================] - 0s - loss: 1.2212 - crps_cost_function: 1.2212 - val_loss: 0.8806 - val_crps_cost_function: 0.8806
Epoch 105/1000
12619/12619 [==============================] - 0s - loss: 1.2171 - crps_cost_function: 1.2171 - val_loss: 0.8771 - val_crps_cost_function: 0.8771
Epoch 106/1000
12619/12619 [==============================] - 0s - loss: 1.2131 - crps_cost_function: 1.2131 - val_loss: 0.8736 - val_crps_cost_function: 0.8736
Epoch 107/1000
12619/12619 [==============================] - 0s - loss: 1.2093 - crps_cost_function: 1.2093 - val_loss: 0.8704 - val_crps_cost_function: 0.8704
Epoch 108/1000
12619/12619 [=====================

12619/12619 [==============================] - 0s - loss: 1.1442 - crps_cost_function: 1.1442 - val_loss: 0.8092 - val_crps_cost_function: 0.8092
Epoch 153/1000
12619/12619 [==============================] - 0s - loss: 1.1438 - crps_cost_function: 1.1438 - val_loss: 0.8088 - val_crps_cost_function: 0.8088
Epoch 154/1000
12619/12619 [==============================] - 0s - loss: 1.1435 - crps_cost_function: 1.1435 - val_loss: 0.8085 - val_crps_cost_function: 0.8085
Epoch 155/1000
12619/12619 [==============================] - 0s - loss: 1.1432 - crps_cost_function: 1.1432 - val_loss: 0.8081 - val_crps_cost_function: 0.8081
Epoch 156/1000
12619/12619 [==============================] - 0s - loss: 1.1429 - crps_cost_function: 1.1429 - val_loss: 0.8078 - val_crps_cost_function: 0.8078
Epoch 157/1000
12619/12619 [==============================] - 0s - loss: 1.1426 - crps_cost_function: 1.1426 - val_loss: 0.8075 - val_crps_cost_function: 0.8075
Epoch 158/1000
12619/12619 [=====================

12619/12619 [==============================] - 0s - loss: 1.1369 - crps_cost_function: 1.1369 - val_loss: 0.8003 - val_crps_cost_function: 0.8003
Epoch 203/1000
12619/12619 [==============================] - 0s - loss: 1.1368 - crps_cost_function: 1.1368 - val_loss: 0.8002 - val_crps_cost_function: 0.8002
Epoch 204/1000
12619/12619 [==============================] - 0s - loss: 1.1367 - crps_cost_function: 1.1367 - val_loss: 0.8001 - val_crps_cost_function: 0.8001
Epoch 205/1000
12619/12619 [==============================] - 0s - loss: 1.1367 - crps_cost_function: 1.1367 - val_loss: 0.8000 - val_crps_cost_function: 0.8000
Epoch 206/1000
12619/12619 [==============================] - 0s - loss: 1.1366 - crps_cost_function: 1.1366 - val_loss: 0.8000 - val_crps_cost_function: 0.8000
Epoch 207/1000
12619/12619 [==============================] - 0s - loss: 1.1366 - crps_cost_function: 1.1366 - val_loss: 0.7999 - val_crps_cost_function: 0.7999
Epoch 208/1000
12619/12619 [=====================

12619/12619 [==============================] - 0s - loss: 1.1346 - crps_cost_function: 1.1346 - val_loss: 0.7965 - val_crps_cost_function: 0.7965
Epoch 253/1000
12619/12619 [==============================] - 0s - loss: 1.1346 - crps_cost_function: 1.1346 - val_loss: 0.7965 - val_crps_cost_function: 0.7965
Epoch 254/1000
12619/12619 [==============================] - 0s - loss: 1.1345 - crps_cost_function: 1.1345 - val_loss: 0.7964 - val_crps_cost_function: 0.7964
Epoch 255/1000
12619/12619 [==============================] - 0s - loss: 1.1345 - crps_cost_function: 1.1345 - val_loss: 0.7963 - val_crps_cost_function: 0.7963
Epoch 256/1000
12619/12619 [==============================] - 0s - loss: 1.1345 - crps_cost_function: 1.1345 - val_loss: 0.7963 - val_crps_cost_function: 0.7963
Epoch 257/1000
12619/12619 [==============================] - 0s - loss: 1.1344 - crps_cost_function: 1.1344 - val_loss: 0.7962 - val_crps_cost_function: 0.7962
Epoch 258/1000
12619/12619 [=====================

Here we see that the CRPS is slightly higher than in our theano implementation. This is most likely due to the differences in the early stopping algorithm.

### Post-processing for 2016

Same as above with the theano model.

In [55]:
date_idx_start = return_date_idx(dates, 2016, 1, 1)
date_idx_stop = return_date_idx(dates, 2016, 12, 31)

In [56]:
model_keras = build_EMOS_Network_keras()
opt = SGD(lr=0.1)  
model_keras.compile(optimizer=opt, loss=crps_cost_function, 
                    metrics=[crps_cost_function])

In [57]:
train_crps_list, valid_crps_list = loop_over_days(
    model_keras,
    tobs_full, 
    tfc_full, 
    date_idx_start, date_idx_stop, 
    window_size=window_size,
    fclt=fclt,     
    epochs_max=steps_max, 
    early_stopping_delta=early_stopping_delta, 
    lr=lr,
    model_type='EMOS_Network_keras',
    verbose=0)

3300
3400
3500
3600
Time: 170.37 s


So the keras implementation is quite a bit slower than the pure theano version. This could be due to the overhead of calling model.fit many many times.

In [58]:
np.mean(train_crps_list), np.mean(valid_crps_list)

(0.98057245693860451, 0.99911120397520148)

The results are very similar to the theano implementation.

### Train 2015, predict 2016

Finally, we will train one single model on all of the 2015 data, and then post-process all of 2016 with this one model. 

In [59]:
tfc_mean_train, tfc_std_train, tobs_train, tfc_mean_test, tfc_std_test, tobs_test = \
        get_train_test_data(tobs_full, tfc_full, date_idx, window_size=365, fclt=0, 
                            subtract_std_mean=False, test_plus=365)

In [60]:
tfc_mean_train.shape, tfc_mean_test.shape

((183957,), (183641,))

In [67]:
model_keras = build_EMOS_Network_keras()
opt = Adam(lr=0.1)  # Adam is a better SGD in a nutshell
model_keras.compile(optimizer=opt, loss=crps_cost_function)

In [69]:
model_keras.fit([tfc_mean_train, tfc_std_train], tobs_train, epochs=10, 
                batch_size=1024, 
                validation_data=[[tfc_mean_test, tfc_std_test], tobs_test])

Train on 183957 samples, validate on 183641 samples
Epoch 1/10
183957/183957 [==============================] - 0s - loss: 1.0866 - val_loss: 1.0910
Epoch 2/10
183957/183957 [==============================] - 0s - loss: 1.0866 - val_loss: 1.1025
Epoch 3/10
183957/183957 [==============================] - 0s - loss: 1.0864 - val_loss: 1.0939
Epoch 4/10
183957/183957 [==============================] - 0s - loss: 1.0862 - val_loss: 1.0924
Epoch 5/10
183957/183957 [==============================] - 0s - loss: 1.0867 - val_loss: 1.0935
Epoch 6/10
183957/183957 [==============================] - 0s - loss: 1.0868 - val_loss: 1.0932
Epoch 7/10
183957/183957 [==============================] - 0s - loss: 1.0866 - val_loss: 1.0966
Epoch 8/10
183957/183957 [==============================] - 0s - loss: 1.0867 - val_loss: 1.1014
Epoch 9/10
183957/183957 [==============================] - 0s - loss: 1.0863 - val_loss: 1.0962
Epoch 10/10
183957/183957 [==============================] - 0s - loss: 1.0

So we get a 2016 CRPS of 1.09 compared to 1.00 for the 25 day rolling window.

## Fully connected model

### For one day

In [5]:
def build_fc_model():
    inp = Input(shape=(2,))
    x = Dense(2, activation='linear')(inp)
    return Model(inputs=inp, outputs=x)

In [125]:
fc_model = build_fc_model()

In [126]:
fc_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 2)                 0         
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


In [127]:
from keras.optimizers import Adam
opt = Adam(lr=0.01)
fc_model.compile(optimizer=opt, loss=crps_cost_function, metrics=[crps_cost_function])

In [128]:
# Now we need to merge the input to a 2D vector
in_train = np.column_stack([tfc_mean_train, tfc_std_train])
in_test = np.column_stack([tfc_mean_test, tfc_std_test])
in_train.shape

(12543, 2)

In [129]:
batch_size = in_train.shape[0]  

In [59]:
# Normalize output
tobs_mean = np.mean(tobs_train)
tobs_std = np.std(tobs_train, ddof=1)
tobs_train_norm = (tobs_train - tobs_mean) / tobs_std
tobs_test_norm = (tobs_test - tobs_mean) / tobs_std

In [130]:
h = fc_model.fit(in_train, tobs_train, epochs=2000, batch_size=batch_size,
          validation_data=[in_test, tobs_test], verbose=1,
          callbacks=[EarlyStopping(monitor='loss', min_delta=1e-6,
                                  patience=2)])

Train on 12543 samples, validate on 494 samples
Epoch 1/2000
12543/12543 [==============================] - 0s - loss: 5.2644 - crps_cost_function: 5.2644 - val_loss: 7.6285 - val_crps_cost_function: 7.6285
Epoch 2/2000
12543/12543 [==============================] - 0s - loss: 5.2173 - crps_cost_function: 5.2173 - val_loss: 7.5632 - val_crps_cost_function: 7.5632
Epoch 3/2000
12543/12543 [==============================] - 0s - loss: 5.1703 - crps_cost_function: 5.1703 - val_loss: 7.4981 - val_crps_cost_function: 7.4981
Epoch 4/2000
12543/12543 [==============================] - 0s - loss: 5.1234 - crps_cost_function: 5.1234 - val_loss: 7.4332 - val_crps_cost_function: 7.4332
Epoch 5/2000
12543/12543 [==============================] - 0s - loss: 5.0767 - crps_cost_function: 5.0767 - val_loss: 7.3684 - val_crps_cost_function: 7.3684
Epoch 6/2000
12543/12543 [==============================] - 0s - loss: 5.0301 - crps_cost_function: 5.0301 - val_loss: 7.3038 - val_crps_cost_function: 7.303

12543/12543 [==============================] - 0s - loss: 2.2602 - crps_cost_function: 2.2602 - val_loss: 3.2722 - val_crps_cost_function: 3.2722
Epoch 103/2000
12543/12543 [==============================] - 0s - loss: 2.2474 - crps_cost_function: 2.2474 - val_loss: 3.2519 - val_crps_cost_function: 3.2519
Epoch 104/2000
12543/12543 [==============================] - 0s - loss: 2.2348 - crps_cost_function: 2.2348 - val_loss: 3.2318 - val_crps_cost_function: 3.2318
Epoch 105/2000
12543/12543 [==============================] - 0s - loss: 2.2223 - crps_cost_function: 2.2223 - val_loss: 3.2121 - val_crps_cost_function: 3.2121
Epoch 106/2000
12543/12543 [==============================] - 0s - loss: 2.2101 - crps_cost_function: 2.2101 - val_loss: 3.1926 - val_crps_cost_function: 3.1926
Epoch 107/2000
12543/12543 [==============================] - 0s - loss: 2.1979 - crps_cost_function: 2.1979 - val_loss: 3.1733 - val_crps_cost_function: 3.1733
Epoch 108/2000
12543/12543 [=====================

12543/12543 [==============================] - 0s - loss: 1.4633 - crps_cost_function: 1.4633 - val_loss: 1.9896 - val_crps_cost_function: 1.9896
Epoch 203/2000
12543/12543 [==============================] - 0s - loss: 1.4581 - crps_cost_function: 1.4581 - val_loss: 1.9811 - val_crps_cost_function: 1.9811
Epoch 204/2000
12543/12543 [==============================] - 0s - loss: 1.4529 - crps_cost_function: 1.4529 - val_loss: 1.9727 - val_crps_cost_function: 1.9727
Epoch 205/2000
12543/12543 [==============================] - 0s - loss: 1.4478 - crps_cost_function: 1.4478 - val_loss: 1.9643 - val_crps_cost_function: 1.9643
Epoch 206/2000
12543/12543 [==============================] - 0s - loss: 1.4426 - crps_cost_function: 1.4426 - val_loss: 1.9560 - val_crps_cost_function: 1.9560
Epoch 207/2000
12543/12543 [==============================] - 0s - loss: 1.4375 - crps_cost_function: 1.4375 - val_loss: 1.9478 - val_crps_cost_function: 1.9478
Epoch 208/2000
12543/12543 [=====================

12543/12543 [==============================] - 0s - loss: 1.0715 - crps_cost_function: 1.0715 - val_loss: 1.3633 - val_crps_cost_function: 1.3633
Epoch 303/2000
12543/12543 [==============================] - 0s - loss: 1.0687 - crps_cost_function: 1.0687 - val_loss: 1.3591 - val_crps_cost_function: 1.3591
Epoch 304/2000
12543/12543 [==============================] - 0s - loss: 1.0660 - crps_cost_function: 1.0660 - val_loss: 1.3551 - val_crps_cost_function: 1.3551
Epoch 305/2000
12543/12543 [==============================] - 0s - loss: 1.0633 - crps_cost_function: 1.0633 - val_loss: 1.3510 - val_crps_cost_function: 1.3510
Epoch 306/2000
12543/12543 [==============================] - 0s - loss: 1.0606 - crps_cost_function: 1.0606 - val_loss: 1.3470 - val_crps_cost_function: 1.3470
Epoch 307/2000
12543/12543 [==============================] - 0s - loss: 1.0580 - crps_cost_function: 1.0580 - val_loss: 1.3430 - val_crps_cost_function: 1.3430
Epoch 308/2000
12543/12543 [=====================

12543/12543 [==============================] - 0s - loss: 0.8787 - crps_cost_function: 0.8787 - val_loss: 1.1454 - val_crps_cost_function: 1.1454
Epoch 403/2000
12543/12543 [==============================] - 0s - loss: 0.8778 - crps_cost_function: 0.8778 - val_loss: 1.1450 - val_crps_cost_function: 1.1450
Epoch 404/2000
12543/12543 [==============================] - 0s - loss: 0.8769 - crps_cost_function: 0.8769 - val_loss: 1.1446 - val_crps_cost_function: 1.1446
Epoch 405/2000
12543/12543 [==============================] - 0s - loss: 0.8760 - crps_cost_function: 0.8760 - val_loss: 1.1443 - val_crps_cost_function: 1.1443
Epoch 406/2000
12543/12543 [==============================] - 0s - loss: 0.8751 - crps_cost_function: 0.8751 - val_loss: 1.1440 - val_crps_cost_function: 1.1440
Epoch 407/2000
12543/12543 [==============================] - 0s - loss: 0.8743 - crps_cost_function: 0.8743 - val_loss: 1.1437 - val_crps_cost_function: 1.1437
Epoch 408/2000
12543/12543 [=====================

...

12543/12543 [==============================] - 0s - loss: 0.8157 - crps_cost_function: 0.8157 - val_loss: 1.0369 - val_crps_cost_function: 1.0369
Epoch 1203/2000
12543/12543 [==============================] - 0s - loss: 0.8156 - crps_cost_function: 0.8156 - val_loss: 1.0368 - val_crps_cost_function: 1.0368
Epoch 1204/2000
12543/12543 [==============================] - 0s - loss: 0.8156 - crps_cost_function: 0.8156 - val_loss: 1.0367 - val_crps_cost_function: 1.0367
Epoch 1205/2000
12543/12543 [==============================] - 0s - loss: 0.8156 - crps_cost_function: 0.8156 - val_loss: 1.0366 - val_crps_cost_function: 1.0366
Epoch 1206/2000
12543/12543 [==============================] - 0s - loss: 0.8156 - crps_cost_function: 0.8156 - val_loss: 1.0366 - val_crps_cost_function: 1.0366
Epoch 1207/2000
12543/12543 [==============================] - 0s - loss: 0.8156 - crps_cost_function: 0.8156 - val_loss: 1.0365 - val_crps_cost_function: 1.0365
Epoch 1208/2000
12543/12543 [===============

12543/12543 [==============================] - 0s - loss: 0.8151 - crps_cost_function: 0.8151 - val_loss: 1.0301 - val_crps_cost_function: 1.0301
Epoch 1303/2000
12543/12543 [==============================] - 0s - loss: 0.8151 - crps_cost_function: 0.8151 - val_loss: 1.0300 - val_crps_cost_function: 1.0300
Epoch 1304/2000
12543/12543 [==============================] - 0s - loss: 0.8151 - crps_cost_function: 0.8151 - val_loss: 1.0300 - val_crps_cost_function: 1.0300
Epoch 1305/2000
12543/12543 [==============================] - 0s - loss: 0.8151 - crps_cost_function: 0.8151 - val_loss: 1.0299 - val_crps_cost_function: 1.0299
Epoch 1306/2000
12543/12543 [==============================] - 0s - loss: 0.8151 - crps_cost_function: 0.8151 - val_loss: 1.0299 - val_crps_cost_function: 1.0299
Epoch 1307/2000
12543/12543 [==============================] - 0s - loss: 0.8151 - crps_cost_function: 0.8151 - val_loss: 1.0298 - val_crps_cost_function: 1.0298
Epoch 1308/2000
12543/12543 [===============

12543/12543 [==============================] - 0s - loss: 0.8148 - crps_cost_function: 0.8148 - val_loss: 1.0248 - val_crps_cost_function: 1.0248
Epoch 1403/2000
12543/12543 [==============================] - 0s - loss: 0.8148 - crps_cost_function: 0.8148 - val_loss: 1.0248 - val_crps_cost_function: 1.0248
Epoch 1404/2000
12543/12543 [==============================] - 0s - loss: 0.8148 - crps_cost_function: 0.8148 - val_loss: 1.0247 - val_crps_cost_function: 1.0247
Epoch 1405/2000
12543/12543 [==============================] - 0s - loss: 0.8148 - crps_cost_function: 0.8148 - val_loss: 1.0247 - val_crps_cost_function: 1.0247
Epoch 1406/2000
12543/12543 [==============================] - 0s - loss: 0.8148 - crps_cost_function: 0.8148 - val_loss: 1.0246 - val_crps_cost_function: 1.0246
Epoch 1407/2000
12543/12543 [==============================] - 0s - loss: 0.8148 - crps_cost_function: 0.8148 - val_loss: 1.0246 - val_crps_cost_function: 1.0246
Epoch 1408/2000
12543/12543 [===============

12543/12543 [==============================] - 0s - loss: 0.8146 - crps_cost_function: 0.8146 - val_loss: 1.0208 - val_crps_cost_function: 1.0208
Epoch 1503/2000
12543/12543 [==============================] - 0s - loss: 0.8146 - crps_cost_function: 0.8146 - val_loss: 1.0208 - val_crps_cost_function: 1.0208
Epoch 1504/2000
12543/12543 [==============================] - 0s - loss: 0.8146 - crps_cost_function: 0.8146 - val_loss: 1.0207 - val_crps_cost_function: 1.0207
Epoch 1505/2000
12543/12543 [==============================] - 0s - loss: 0.8146 - crps_cost_function: 0.8146 - val_loss: 1.0207 - val_crps_cost_function: 1.0207
Epoch 1506/2000
12543/12543 [==============================] - 0s - loss: 0.8146 - crps_cost_function: 0.8146 - val_loss: 1.0207 - val_crps_cost_function: 1.0207
Epoch 1507/2000
12543/12543 [==============================] - 0s - loss: 0.8146 - crps_cost_function: 0.8146 - val_loss: 1.0206 - val_crps_cost_function: 1.0206
Epoch 1508/2000
12543/12543 [===============

12543/12543 [==============================] - 0s - loss: 0.8145 - crps_cost_function: 0.8145 - val_loss: 1.0178 - val_crps_cost_function: 1.0178
Epoch 1603/2000
12543/12543 [==============================] - 0s - loss: 0.8145 - crps_cost_function: 0.8145 - val_loss: 1.0178 - val_crps_cost_function: 1.0178
Epoch 1604/2000
12543/12543 [==============================] - 0s - loss: 0.8145 - crps_cost_function: 0.8145 - val_loss: 1.0178 - val_crps_cost_function: 1.0178
Epoch 1605/2000
12543/12543 [==============================] - 0s - loss: 0.8145 - crps_cost_function: 0.8145 - val_loss: 1.0177 - val_crps_cost_function: 1.0177
Epoch 1606/2000
12543/12543 [==============================] - 0s - loss: 0.8145 - crps_cost_function: 0.8145 - val_loss: 1.0177 - val_crps_cost_function: 1.0177
Epoch 1607/2000
12543/12543 [==============================] - 0s - loss: 0.8145 - crps_cost_function: 0.8145 - val_loss: 1.0177 - val_crps_cost_function: 1.0177
Epoch 1608/2000
12543/12543 [===============

In [68]:
preds = fc_model.predict(in_test)

In [69]:
preds_conv = preds * tobs_std + tobs_mean
preds[:5], preds_conv[:5]

(array([[ 1.823313  , -0.13283765],
        [ 0.85769427, -0.70848733],
        [ 0.14632295,  0.54326606],
        [ 1.92873287, -0.44145757],
        [ 1.13935816, -0.04086858]], dtype=float32),
 array([[ 8.8684082 ,  2.88078165],
        [ 5.91272354,  1.11876225],
        [ 3.73527098,  4.95028305],
        [ 9.19109058,  1.93611991],
        [ 6.77487564,  3.162292  ]], dtype=float32))

In [74]:
tobs_test_norm[:5], tobs_test[:5]

(array([ 1.76829398,  1.4415952 , -0.68194705,  1.8989737 ,  1.50693524], dtype=float32),
 array([ 8.69999981,  7.69999981,  1.20000005,  9.10000038,  7.9000001 ], dtype=float32))

In [70]:
preds_conv.shape

(494, 2)

In [75]:
crps_normal(preds[:, 0], preds[:, 1], tobs_test_norm)[:5]

array([-0.04000679, -0.34737915,  0.55186327, -0.10396643, -0.34451946])

In [309]:
fc_model.evaluate(in_train, tobs_train, batch_size=64)

159680/176948 [==========================>...] - ETA: 0s

[nan, nan]

In [73]:
fc_model.evaluate(in_test, tobs_test_norm, batch_size=64)

 64/494 [==>...........................] - ETA: 0s

[0.3456875362589566, 0.3456875362589566]

In [54]:
fc_model.layers[-1].get_weights()

[array([[ nan,  nan],
        [ nan,  nan]], dtype=float32), array([ nan,  nan], dtype=float32)]

### Train for the entire period

In [ ]:
fc_model = build_fc_model()
opt = SGD(lr=0.1)  
fc_model.compile(optimizer=opt, loss=crps_cost_function, 
                 metrics=[crps_cost_function])

In [ ]:
train_crps_list, valid_crps_list = loop_over_days(
    fc_model,
    tobs_full, 
    tfc_full, 
    date_idx_start, date_idx_stop, 
    window_size=window_size,
    fclt=fclt,     
    epochs_max=steps_max, 
    early_stopping_delta=early_stopping_delta, 
    lr=lr,
    model_type='fc_model',
    verbose=0)

In [ ]:
np.mean(valid_crps_list)

## With hidden layers

In [10]:
def build_hidden_model(hidden_nodes):
    inp = Input(shape=(2,))
    x = Dense(hidden_nodes, activation='relu')(inp)
    x = Dense(2, activation='linear')(x)
    return Model(inputs=inp, outputs=x)

In [40]:
hidden_model = build_hidden_model(100)

In [41]:
hidden_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               300       
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 202       
Total params: 502
Trainable params: 502
Non-trainable params: 0
_________________________________________________________________


In [42]:
from keras.optimizers import Adam
opt = Adam(lr=0.01)
hidden_model.compile(optimizer=opt, loss=crps_cost_function, metrics=[crps_cost_function])

In [43]:
in_train = np.column_stack([tfc_mean_train, tfc_std_train])
in_test = np.column_stack([tfc_mean_test, tfc_std_test])
batch_size = in_train.shape[0] 

In [44]:
h = hidden_model.fit(in_train, tobs_train, epochs=2000, batch_size=batch_size,
          validation_data=[in_test, tobs_test], verbose=1)

Train on 12543 samples, validate on 494 samples
Epoch 1/2000
12543/12543 [==============================] - 0s - loss: 3.5495 - crps_cost_function: 3.5495 - val_loss: 4.7025 - val_crps_cost_function: 4.7025
Epoch 2/2000
12543/12543 [==============================] - 0s - loss: 3.2016 - crps_cost_function: 3.2016 - val_loss: 4.2377 - val_crps_cost_function: 4.2377
Epoch 3/2000
12543/12543 [==============================] - 0s - loss: 2.8734 - crps_cost_function: 2.8734 - val_loss: 3.8006 - val_crps_cost_function: 3.8006
Epoch 4/2000
12543/12543 [==============================] - 0s - loss: 2.5690 - crps_cost_function: 2.5690 - val_loss: 3.3962 - val_crps_cost_function: 3.3962
Epoch 5/2000
12543/12543 [==============================] - 0s - loss: 2.2892 - crps_cost_function: 2.2892 - val_loss: 3.0293 - val_crps_cost_function: 3.0293
Epoch 6/2000
12543/12543 [==============================] - 0s - loss: 2.0373 - crps_cost_function: 2.0373 - val_loss: 2.7036 - val_crps_cost_function: 2.703

12543/12543 [==============================] - 0s - loss: 0.8072 - crps_cost_function: 0.8072 - val_loss: 0.9793 - val_crps_cost_function: 0.9793
Epoch 103/2000
12543/12543 [==============================] - 0s - loss: 0.8071 - crps_cost_function: 0.8071 - val_loss: 0.9816 - val_crps_cost_function: 0.9816
Epoch 104/2000
12543/12543 [==============================] - 0s - loss: 0.8070 - crps_cost_function: 0.8070 - val_loss: 0.9834 - val_crps_cost_function: 0.9834
Epoch 105/2000
12543/12543 [==============================] - 0s - loss: 0.8069 - crps_cost_function: 0.8069 - val_loss: 0.9845 - val_crps_cost_function: 0.9845
Epoch 106/2000
12543/12543 [==============================] - 0s - loss: 0.8068 - crps_cost_function: 0.8068 - val_loss: 0.9849 - val_crps_cost_function: 0.9849
Epoch 107/2000
12543/12543 [==============================] - 0s - loss: 0.8067 - crps_cost_function: 0.8067 - val_loss: 0.9847 - val_crps_cost_function: 0.9847
Epoch 108/2000
12543/12543 [=====================

12543/12543 [==============================] - 0s - loss: 0.8038 - crps_cost_function: 0.8038 - val_loss: 0.9957 - val_crps_cost_function: 0.9957
Epoch 203/2000
12543/12543 [==============================] - 0s - loss: 0.8038 - crps_cost_function: 0.8038 - val_loss: 0.9958 - val_crps_cost_function: 0.9958
Epoch 204/2000
12543/12543 [==============================] - 0s - loss: 0.8038 - crps_cost_function: 0.8038 - val_loss: 0.9959 - val_crps_cost_function: 0.9959
Epoch 205/2000
12543/12543 [==============================] - 0s - loss: 0.8038 - crps_cost_function: 0.8038 - val_loss: 0.9960 - val_crps_cost_function: 0.9960
Epoch 206/2000
12543/12543 [==============================] - 0s - loss: 0.8037 - crps_cost_function: 0.8037 - val_loss: 0.9961 - val_crps_cost_function: 0.9961
Epoch 207/2000
12543/12543 [==============================] - 0s - loss: 0.8037 - crps_cost_function: 0.8037 - val_loss: 0.9961 - val_crps_cost_function: 0.9961
Epoch 208/2000
12543/12543 [=====================

12543/12543 [==============================] - 0s - loss: 0.8029 - crps_cost_function: 0.8029 - val_loss: 1.0022 - val_crps_cost_function: 1.0022
Epoch 303/2000
12543/12543 [==============================] - 0s - loss: 0.8029 - crps_cost_function: 0.8029 - val_loss: 1.0023 - val_crps_cost_function: 1.0023
Epoch 304/2000
12543/12543 [==============================] - 0s - loss: 0.8028 - crps_cost_function: 0.8028 - val_loss: 1.0023 - val_crps_cost_function: 1.0023
Epoch 305/2000
12543/12543 [==============================] - 0s - loss: 0.8028 - crps_cost_function: 0.8028 - val_loss: 1.0024 - val_crps_cost_function: 1.0024
Epoch 306/2000
12543/12543 [==============================] - 0s - loss: 0.8028 - crps_cost_function: 0.8028 - val_loss: 1.0025 - val_crps_cost_function: 1.0025
Epoch 307/2000
12543/12543 [==============================] - 0s - loss: 0.8028 - crps_cost_function: 0.8028 - val_loss: 1.0025 - val_crps_cost_function: 1.0025
Epoch 308/2000
12543/12543 [=====================

12543/12543 [==============================] - 0s - loss: 0.8018 - crps_cost_function: 0.8018 - val_loss: 1.0065 - val_crps_cost_function: 1.0065
Epoch 403/2000
12543/12543 [==============================] - 0s - loss: 0.8018 - crps_cost_function: 0.8018 - val_loss: 1.0066 - val_crps_cost_function: 1.0066
Epoch 404/2000
12543/12543 [==============================] - 0s - loss: 0.8018 - crps_cost_function: 0.8018 - val_loss: 1.0066 - val_crps_cost_function: 1.0066
Epoch 405/2000
12543/12543 [==============================] - 0s - loss: 0.8018 - crps_cost_function: 0.8018 - val_loss: 1.0066 - val_crps_cost_function: 1.0066
Epoch 406/2000
12543/12543 [==============================] - 0s - loss: 0.8018 - crps_cost_function: 0.8018 - val_loss: 1.0067 - val_crps_cost_function: 1.0067
Epoch 407/2000
12543/12543 [==============================] - 0s - loss: 0.8018 - crps_cost_function: 0.8018 - val_loss: 1.0067 - val_crps_cost_function: 1.0067
Epoch 408/2000
12543/12543 [=====================

...

12543/12543 [==============================] - 0s - loss: 0.7945 - crps_cost_function: 0.7945 - val_loss: 1.0209 - val_crps_cost_function: 1.0209
Epoch 1503/2000
12543/12543 [==============================] - 0s - loss: 0.7945 - crps_cost_function: 0.7945 - val_loss: 1.0212 - val_crps_cost_function: 1.0212
Epoch 1504/2000
12543/12543 [==============================] - 0s - loss: 0.7945 - crps_cost_function: 0.7945 - val_loss: 1.0208 - val_crps_cost_function: 1.0208
Epoch 1505/2000
12543/12543 [==============================] - 0s - loss: 0.7945 - crps_cost_function: 0.7945 - val_loss: 1.0212 - val_crps_cost_function: 1.0212
Epoch 1506/2000
12543/12543 [==============================] - 0s - loss: 0.7945 - crps_cost_function: 0.7945 - val_loss: 1.0211 - val_crps_cost_function: 1.0211
Epoch 1507/2000
12543/12543 [==============================] - 0s - loss: 0.7945 - crps_cost_function: 0.7945 - val_loss: 1.0212 - val_crps_cost_function: 1.0212
Epoch 1508/2000
12543/12543 [===============

12543/12543 [==============================] - 0s - loss: 0.7940 - crps_cost_function: 0.7940 - val_loss: 1.0300 - val_crps_cost_function: 1.0300
Epoch 1603/2000
12543/12543 [==============================] - 0s - loss: 0.7939 - crps_cost_function: 0.7939 - val_loss: 1.0156 - val_crps_cost_function: 1.0156
Epoch 1604/2000
12543/12543 [==============================] - 0s - loss: 0.7939 - crps_cost_function: 0.7939 - val_loss: 1.0274 - val_crps_cost_function: 1.0274
Epoch 1605/2000
12543/12543 [==============================] - 0s - loss: 0.7939 - crps_cost_function: 0.7939 - val_loss: 1.0185 - val_crps_cost_function: 1.0185
Epoch 1606/2000
12543/12543 [==============================] - 0s - loss: 0.7938 - crps_cost_function: 0.7938 - val_loss: 1.0242 - val_crps_cost_function: 1.0242
Epoch 1607/2000
12543/12543 [==============================] - 0s - loss: 0.7938 - crps_cost_function: 0.7938 - val_loss: 1.0216 - val_crps_cost_function: 1.0216
Epoch 1608/2000
12543/12543 [===============

12543/12543 [==============================] - 0s - loss: 0.7933 - crps_cost_function: 0.7933 - val_loss: 1.0329 - val_crps_cost_function: 1.0329
Epoch 1703/2000
12543/12543 [==============================] - 0s - loss: 0.7933 - crps_cost_function: 0.7933 - val_loss: 1.0137 - val_crps_cost_function: 1.0137
Epoch 1704/2000
12543/12543 [==============================] - 0s - loss: 0.7933 - crps_cost_function: 0.7933 - val_loss: 1.0346 - val_crps_cost_function: 1.0346
Epoch 1705/2000
12543/12543 [==============================] - 0s - loss: 0.7933 - crps_cost_function: 0.7933 - val_loss: 1.0124 - val_crps_cost_function: 1.0124
Epoch 1706/2000
12543/12543 [==============================] - 0s - loss: 0.7934 - crps_cost_function: 0.7934 - val_loss: 1.0362 - val_crps_cost_function: 1.0362
Epoch 1707/2000
12543/12543 [==============================] - 0s - loss: 0.7934 - crps_cost_function: 0.7934 - val_loss: 1.0118 - val_crps_cost_function: 1.0118
Epoch 1708/2000
12543/12543 [===============

12543/12543 [==============================] - 0s - loss: 0.7928 - crps_cost_function: 0.7928 - val_loss: 1.0346 - val_crps_cost_function: 1.0346
Epoch 1803/2000
12543/12543 [==============================] - 0s - loss: 0.7927 - crps_cost_function: 0.7927 - val_loss: 1.0231 - val_crps_cost_function: 1.0231
Epoch 1804/2000
12543/12543 [==============================] - 0s - loss: 0.7926 - crps_cost_function: 0.7926 - val_loss: 1.0241 - val_crps_cost_function: 1.0241
Epoch 1805/2000
12543/12543 [==============================] - 0s - loss: 0.7926 - crps_cost_function: 0.7926 - val_loss: 1.0326 - val_crps_cost_function: 1.0326
Epoch 1806/2000
12543/12543 [==============================] - 0s - loss: 0.7926 - crps_cost_function: 0.7926 - val_loss: 1.0169 - val_crps_cost_function: 1.0169
Epoch 1807/2000
12543/12543 [==============================] - 0s - loss: 0.7927 - crps_cost_function: 0.7927 - val_loss: 1.0359 - val_crps_cost_function: 1.0359
Epoch 1808/2000
12543/12543 [===============

12543/12543 [==============================] - 0s - loss: 0.7922 - crps_cost_function: 0.7922 - val_loss: 1.0321 - val_crps_cost_function: 1.0321
Epoch 1903/2000
12543/12543 [==============================] - 0s - loss: 0.7921 - crps_cost_function: 0.7921 - val_loss: 1.0258 - val_crps_cost_function: 1.0258
Epoch 1904/2000
12543/12543 [==============================] - 0s - loss: 0.7921 - crps_cost_function: 0.7921 - val_loss: 1.0282 - val_crps_cost_function: 1.0282
Epoch 1905/2000
12543/12543 [==============================] - 0s - loss: 0.7921 - crps_cost_function: 0.7921 - val_loss: 1.0297 - val_crps_cost_function: 1.0297
Epoch 1906/2000
12543/12543 [==============================] - 0s - loss: 0.7921 - crps_cost_function: 0.7921 - val_loss: 1.0250 - val_crps_cost_function: 1.0250
Epoch 1907/2000
12543/12543 [==============================] - 0s - loss: 0.7921 - crps_cost_function: 0.7921 - val_loss: 1.0323 - val_crps_cost_function: 1.0323
Epoch 1908/2000
12543/12543 [===============

## Get auxiliary data

### Check out dataset

In [27]:
aux_dir = DATA_DIR + 'auxiliary/interpolated_to_stations/'

In [29]:
DATA_DIR + 'data_interpolated_00UTC.nc'

'/Volumes/STICK/data/ppnn_data/data_interpolated_00UTC.nc'

In [30]:
# Load all the data
target = Dataset(DATA_DIR + 'data_interpolated_00UTC.nc').variables['t2m_obs'][:]
target.shape   # time, station

(3653, 537)

In [31]:
t2m_obs = Dataset(DATA_DIR + 'data_interpolated_00UTC.nc').variables['t2m_fc'][:]
t2m_obs.shape   # time, member, station

(3653, 50, 537)

In [32]:
geo = Dataset(aux_dir + 'data_aux_geo_interpolated.nc')
geo

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): station(537), nchar(36)
    variables(dimensions): int32 station(station), float32 lsm(station), float32 orog(station), float32 station_alt(station), float32 station_lat(station), float32 station_lon(station), float32 station_id(station), |S1 station_loc(station,nchar)
    groups: 

In [38]:
geo.variables['orog'][:5]

masked_array(data = [107.43946075439453 19.39777374267578 604.885009765625 461.4418029785156
 47.63252258300781],
             mask = [False False False False False],
       fill_value = nan)

In [54]:
a = geo.variables['station_alt'][:]

In [56]:
a.shape

(537,)

In [60]:
np.array([a] * 10).shape

(10, 537)

In [45]:
pl500 = Dataset(aux_dir + 'data_aux_pl500_interpolated_00UTC.nc')
pl500

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): station(537), member(50), time(3653), nchar(36)
    variables(dimensions): float64 station(station), int32 member(member), int32 time(time), float32 u_pl500_fc(time,member,station), float32 v_pl500_fc(time,member,station), float32 gh_pl500_fc(time,member,station), float32 station_alt(station), float32 station_lat(station), float32 station_lon(station), float32 station_id(station), |S1 station_loc(station,nchar)
    groups: 

In [46]:
pl850 = Dataset(aux_dir + 'data_aux_pl850_interpolated_00UTC.nc')
pl850

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): station(537), member(50), time(3653), nchar(36)
    variables(dimensions): float64 station(station), int32 member(member), int32 time(time), float32 u_pl850_fc(time,member,station), float32 v_pl850_fc(time,member,station), float32 q_pl850_fc(time,member,station), float32 station_alt(station), float32 station_lat(station), float32 station_lon(station), float32 station_id(station), |S1 station_loc(station,nchar)
    groups: 

In [49]:
pl850.variables['q_pl850_fc']

<class 'netCDF4._netCDF4.Variable'>
float32 q_pl850_fc(time, member, station)
    units: kg kg^-1
    _FillValue: nan
    long_name: interpolated specific humidity at pressure level 850 hPa ensemble forecast
unlimited dimensions: 
current shape = (3653, 50, 537)
filling on

In [51]:
surf = Dataset(aux_dir + 'data_aux_surface_interpolated_00UTC.nc')
surf

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): station(537), member(50), time(3653), nchar(36)
    variables(dimensions): float64 station(station), int32 member(member), int32 time(time), float32 cape_fc(time,member,station), float32 sp_fc(time,member,station), float32 tcc_fc(time,member,station), float32 station_alt(station), float32 station_lat(station), float32 station_lon(station), float32 station_id(station), |S1 station_loc(station,nchar)
    groups: 

In [52]:
surf.variables['sp_fc']

<class 'netCDF4._netCDF4.Variable'>
float32 sp_fc(time, member, station)
    units: Pa
    _FillValue: nan
    long_name: interpolated surface pressure ensemble forecast
unlimited dimensions: 
current shape = (3653, 50, 537)
filling on

In [53]:
surf.variables['tcc_fc']

<class 'netCDF4._netCDF4.Variable'>
float32 tcc_fc(time, member, station)
    units: %
    _FillValue: nan
    long_name: interpolated total cloud cover ensemble forecast
unlimited dimensions: 
current shape = (3653, 50, 537)
filling on

### Data loading and preparing functions
Raw 
- Input: Several files with the dataset for all times and stations. Some have dimensions [time, mem, station], some have only station as a dimension.
    - The ensemble files have to be converted to mean and std
    - The features have to be normalized, but how? I probably should do min-max scaling to a range of [0, 1]
- Output: t2m forecasts in one file, but with missing values.

The converted data should be 


Current variables:


**Target:** 
- t2m_fc in data_interpolated.nc [time, station]

**Features:**
- data_interpolated.nc [time, mem, station]
    - t2m_fc
- data_aux_geo_interpolated.nc [station]
    - lsm (Land-sea mask, all 1 for station data, can leave out!)
    - orog (Don't know what that is...)
    - station_alt (Altitude)
    - station_lat
    - station_lon
- data_aux_pl500_interpolated_00UTC.nc [time, mem, station]
    - u_pl500_fc/u_pl500_fc (Wind)
    - gh_pl500_fc (geopotential height)
- data_aux_pl850_interpolated_00UTC.nc [time, mem, station]
    - u_pl850_fc/u_pl850_fc (Wind)
    - q_pl850_fc (Humidity)
- data_aux_surface_interpolated_00UTC.nc [time, mem, station]
    - cape_fc (CAPE)
    - sp_fc (Surface pressure)
    - tcc_fc (Cloud cover)
    
The preprocessed data should have dimensions [time, station, feature] so that I can easily split it then. Missing values we will deal with later!

In [69]:
def prepare_data(data_dir):
    """
    
    """
    aux_dir = data_dir + 'auxiliary/interpolated_to_stations/'
    
    fl = []   # Here we will store all the features
    # Load Temperature data
    rg = Dataset(data_dir + 'data_interpolated_00UTC.nc')
    target = rg.variables['t2m_obs'][:]
    ntime = target.shape[0]
    dates = num2date(rg.variables['time'][:],
                     units='seconds since 1970-01-01 00:00 UTC')
    fl.append(np.mean(rg.variables['t2m_fc'][:], axis=1))
    fl.append(np.std(rg.variables['t2m_fc'][:], axis=1, ddof=1))
    rg.close()
    
    # Load geo data
    # Here we have to duplicate this to all timesteps
    rg = Dataset(aux_dir + 'data_aux_geo_interpolated.nc')
    fl.append(np.array([rg.variables['orog'][:]] * ntime))
    fl.append(np.array([rg.variables['station_alt'][:]] * ntime))
    fl.append(np.array([rg.variables['station_lat'][:]] * ntime))
    fl.append(np.array([rg.variables['station_lon'][:]] * ntime))
    rg.close()
    
    # Load pl500 data
    rg = Dataset(aux_dir + 'data_aux_pl500_interpolated_00UTC.nc')
    fl.append(np.mean(rg.variables['u_pl500_fc'][:], axis=1))
    fl.append(np.std(rg.variables['u_pl500_fc'][:], axis=1, ddof=1))
    fl.append(np.mean(rg.variables['v_pl500_fc'][:], axis=1))
    fl.append(np.std(rg.variables['v_pl500_fc'][:], axis=1, ddof=1))
    fl.append(np.mean(rg.variables['gh_pl500_fc'][:], axis=1))
    fl.append(np.std(rg.variables['gh_pl500_fc'][:], axis=1, ddof=1))
    rg.close()
    
    # Load pl850 data
    rg = Dataset(aux_dir + 'data_aux_pl850_interpolated_00UTC.nc')
    fl.append(np.mean(rg.variables['u_pl850_fc'][:], axis=1))
    fl.append(np.std(rg.variables['u_pl850_fc'][:], axis=1, ddof=1))
    fl.append(np.mean(rg.variables['v_pl850_fc'][:], axis=1))
    fl.append(np.std(rg.variables['v_pl850_fc'][:], axis=1, ddof=1))
    fl.append(np.mean(rg.variables['q_pl850_fc'][:], axis=1))
    fl.append(np.std(rg.variables['q_pl850_fc'][:], axis=1, ddof=1))
    rg.close()
    
    # Load surface data
    rg = Dataset(aux_dir + 'data_aux_surface_interpolated_00UTC.nc')
    fl.append(np.mean(rg.variables['cape_fc'][:], axis=1))
    fl.append(np.std(rg.variables['cape_fc'][:], axis=1, ddof=1))
    fl.append(np.mean(rg.variables['sp_fc'][:], axis=1))
    fl.append(np.std(rg.variables['sp_fc'][:], axis=1, ddof=1))
    fl.append(np.mean(rg.variables['tcc_fc'][:], axis=1))
    fl.append(np.std(rg.variables['tcc_fc'][:], axis=1, ddof=1))
    rg.close()
    
    return target, np.array(fl), dates

In [70]:
target, features, dates = prepare_data(DATA_DIR)

In [71]:
features.shape

(24, 3653, 537)

In [216]:
id_array = np.array([np.arange(537)] * 3653).reshape((1, 3653, 537))
id_array.shape

(1, 3653, 537)

In [219]:
features_id = np.concatenate([features, id_array])
features_id.shape

(25, 3653, 537)

In [123]:
np.mean(features, axis=(1,2))

array([  7.20602330e+00,   8.44141490e-01,   2.95234511e+02,
         2.89700186e+02,   5.08126088e+01,   1.01840399e+01,
         8.56238335e+00,   2.65909153e+00,  -5.75152970e-01,
         2.75810911e+00,   5.59783297e+03,   1.30869580e+01,
         4.65902534e+00,   2.06335589e+00,   4.36603520e-01,
         1.77180366e+00,   4.28229534e-03,   6.04551613e-04,
         1.91571153e+01,   1.84251334e+01,   9.81000748e+04,
         1.15540407e+02,   6.16691684e+01,   2.09259718e+01])

In [124]:
np.std(features, axis=(1,2))

array([  6.83739609e+00,   3.82234452e-01,   2.34161657e+02,
         2.78959442e+02,   1.91586392e+00,   2.08465193e+00,
         1.09146544e+01,   1.14510223e+00,   1.09156339e+01,
         1.37481028e+00,   1.54118379e+02,   6.53550690e+00,
         7.68199681e+00,   8.93665720e-01,   5.26589241e+00,
         8.19539609e-01,   2.18533165e-03,   3.68541141e-04,
         8.26492572e+01,   5.58905364e+01,   2.81062251e+03,
         5.07310894e+01,   3.18192377e+01,   1.15815637e+01])

In [110]:
date_idx_start, date_idx_stop

(3285, 3650)

In [221]:
def scale_and_split(target, features, date_idx, period, return_id_array=False): 
        
    features_train = features[:, date_idx-period:date_idx]
    target_train = target[date_idx-period:date_idx]
    features_test = features[:, date_idx:date_idx+period]
    target_test = target[date_idx:date_idx+period]
    # Ravel arrays
    features_train = np.reshape(features_train, (features_train.shape[0], -1))
    target_train = np.reshape(target_train, (-1))
    features_test = np.reshape(features_test, (features_train.shape[0], -1))
    target_test = np.reshape(target_test, (-1))
    # Remove nans
    train_mask = np.isfinite(target_train.data)
    features_train = features_train[:, train_mask]
    target_train = target_train.data[train_mask]
    test_mask = np.isfinite(target_test.data)
    features_test = features_test[:, test_mask]
    target_test = target_test.data[test_mask]
    features_train = np.rollaxis(features_train, 1, 0)
    features_test = np.rollaxis(features_test, 1, 0)
    # Scale features
    features_max = np.max(features_train, axis=0)
    target_max = np.max(target_train)
    features_train /= features_max
    features_test /= features_max
    # target_train /= target_max
    # target_test /= target_max
    
    if return_id_array:
        s = features.shape
        id_array = np.array([np.arange(s[-1])] * s[1])
        id_array_train = id_array[date_idx-period:date_idx]
        id_array_test = id_array[date_idx:date_idx+period]
        id_array_train = np.reshape(id_array_train, (-1))
        id_array_test = np.reshape(id_array_test, (-1))
        id_array_train = id_array_train[train_mask]
        id_array_test = id_array_test[test_mask]
        
        return features_train, target_train, features_test, target_test, id_array_train, id_array_test
    else:
        return features_train, target_train, features_test, target_test

In [209]:
features_train, target_train, features_test, target_test = scale_and_split(target, features, date_idx_start, 365)

In [210]:
features_train.std(axis=0)

array([ 0.25075446,  0.09661028,  0.18430265,  0.0927913 ,  0.03468791,
        0.13954662,  0.2262587 ,  0.07106561,  0.30289865,  0.08776971,
        0.02569923,  0.09980712,  0.23046334,  0.06643576,  0.20091854,
        0.08406755,  0.19337898,  0.11743256,  0.05347628,  0.06230706,
        0.02710518,  0.08177471,  0.3189943 ,  0.23367659])

In [211]:
target_train

array([ 2.20000005, -3.70000005, -2.0999999 , ...,  7.5       ,
        0.40000001, -0.40000001], dtype=float32)

### Test with hidden layer model

In [194]:
def build_hidden_model2(input_shape, hidden_nodes):
    inp = Input(shape=(input_shape,))
    x = Dense(hidden_nodes, activation='relu')(inp)
    x = Dense(2, activation='linear')(x)
    model = Model(inputs=inp, outputs=x)
    opt = Adam(lr=0.001)
    model.compile(optimizer=opt, loss=crps_cost_function, 
                  metrics=[crps_cost_function])
    return model

In [195]:
hidden_model = build_hidden_model2(features_train.shape[1], 100)

In [196]:
hidden_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 24)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 100)               2500      
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 202       
Total params: 2,702
Trainable params: 2,702
Non-trainable params: 0
_________________________________________________________________


In [199]:
hidden_model.fit(features_train, target_train, epochs=10, batch_size=1024,
                 validation_data=[features_test, target_test], verbose=1)

Train on 180849 samples, validate on 181718 samples
Epoch 1/10
180849/180849 [==============================] - 1s - loss: 0.9484 - crps_cost_function: 0.9484 - val_loss: 0.9240 - val_crps_cost_function: 0.9240
Epoch 2/10
180849/180849 [==============================] - 1s - loss: 0.9448 - crps_cost_function: 0.9448 - val_loss: 0.9234 - val_crps_cost_function: 0.9234
Epoch 3/10
180849/180849 [==============================] - 1s - loss: 0.9410 - crps_cost_function: 0.9410 - val_loss: 0.9255 - val_crps_cost_function: 0.9255
Epoch 4/10
180849/180849 [==============================] - 1s - loss: 0.9381 - crps_cost_function: 0.9381 - val_loss: 0.9202 - val_crps_cost_function: 0.9202
Epoch 5/10
180849/180849 [==============================] - 1s - loss: 0.9357 - crps_cost_function: 0.9357 - val_loss: 0.9214 - val_crps_cost_function: 0.9214
Epoch 6/10
180849/180849 [==============================] - 1s - loss: 0.9339 - crps_cost_function: 0.9339 - val_loss: 0.9209 - val_crps_cost_function: 0

In [201]:
# More complex model
hidden_model = build_hidden_model2(features_train.shape[1], 1000)

In [202]:
hidden_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 24)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 1000)              25000     
_________________________________________________________________
dense_22 (Dense)             (None, 2)                 2002      
Total params: 27,002
Trainable params: 27,002
Non-trainable params: 0
_________________________________________________________________


In [203]:
hidden_model.fit(features_train, target_train, epochs=10, batch_size=1024,
                 validation_data=[features_test, target_test], verbose=1)

Train on 180849 samples, validate on 181718 samples
Epoch 1/10
180849/180849 [==============================] - 9s - loss: 2.7829 - crps_cost_function: 2.7829 - val_loss: 1.2553 - val_crps_cost_function: 1.2553
Epoch 2/10
180849/180849 [==============================] - 9s - loss: 1.0573 - crps_cost_function: 1.0573 - val_loss: 1.0045 - val_crps_cost_function: 1.0045
Epoch 3/10
180849/180849 [==============================] - 9s - loss: 0.9902 - crps_cost_function: 0.9902 - val_loss: 0.9644 - val_crps_cost_function: 0.9644
Epoch 4/10
180849/180849 [==============================] - 9s - loss: 0.9689 - crps_cost_function: 0.9689 - val_loss: 0.9436 - val_crps_cost_function: 0.9436
Epoch 5/10
180849/180849 [==============================] - 9s - loss: 0.9588 - crps_cost_function: 0.9588 - val_loss: 0.9399 - val_crps_cost_function: 0.9399
Epoch 6/10
180849/180849 [==============================] - 9s - loss: 0.9535 - crps_cost_function: 0.9535 - val_loss: 0.9307 - val_crps_cost_function: 0

### Try out an embedding

In [222]:
# Add continuous station ID
features_train, target_train, features_test, target_test, id_array_train, id_array_test = scale_and_split(target, features, date_idx_start, 365, return_id_array=True)

In [247]:
def build_embedding_model(features_shape, max_id, emb_size, hidden_nodes):
    features_in = Input(shape=(features_shape,))
    id_in = Input(shape=(1,))
    emb = Embedding(max_id + 1, emb_size)(id_in)
    emb = Flatten()(emb)
    x = merge([features_in, emb], mode='concat')
    x = Dense(hidden_nodes, activation='relu')(x)
    x = Dense(2, activation='linear')(x)
    model = Model(inputs=[features_in, id_in], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(optimizer=opt, loss=crps_cost_function, 
                  metrics=[crps_cost_function])
    return model

In [251]:
emb_model = build_embedding_model(features_train.shape[1], 
                                  537, 
                                  5, 100)

/Users/stephanrasp/anaconda/envs/py36_keras/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/Users/stephanrasp/anaconda/envs/py36_keras/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [252]:
emb_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_32 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_8 (Embedding)          (None, 1, 5)          2690        input_32[0][0]                   
____________________________________________________________________________________________________
input_31 (InputLayer)            (None, 24)            0                                            
____________________________________________________________________________________________________
flatten_6 (Flatten)              (None, 5)             0           embedding_8[0][0]                
___________________________________________________________________________________________

In [254]:
emb_model.fit([features_train, id_array_train], target_train, epochs=10, batch_size=1024,
                 validation_data=[[features_test, id_array_test], target_test], verbose=1)

Train on 180849 samples, validate on 181718 samples
Epoch 1/10
180849/180849 [==============================] - 1s - loss: 0.9080 - crps_cost_function: 0.9080 - val_loss: 0.8879 - val_crps_cost_function: 0.8879
Epoch 2/10
180849/180849 [==============================] - 1s - loss: 0.9003 - crps_cost_function: 0.9003 - val_loss: 0.8840 - val_crps_cost_function: 0.8840
Epoch 3/10
180849/180849 [==============================] - 1s - loss: 0.8904 - crps_cost_function: 0.8904 - val_loss: 0.8805 - val_crps_cost_function: 0.8805
Epoch 4/10
180849/180849 [==============================] - 1s - loss: 0.8815 - crps_cost_function: 0.8815 - val_loss: 0.8751 - val_crps_cost_function: 0.8751
Epoch 5/10
180849/180849 [==============================] - 1s - loss: 0.8753 - crps_cost_function: 0.8753 - val_loss: 0.8733 - val_crps_cost_function: 0.8733
Epoch 6/10
180849/180849 [==============================] - 1s - loss: 0.8714 - crps_cost_function: 0.8714 - val_loss: 0.8673 - val_crps_cost_function: 0

In [261]:
def build_embedding_model2(features_shape, max_id, emb_size, hidden_nodes):
    features_in = Input(shape=(features_shape,))
    id_in = Input(shape=(1,))
    emb = Embedding(max_id + 1, emb_size)(id_in)
    emb = Flatten()(emb)
    x = merge([features_in, emb], mode='concat')
    x = Dropout(0.25)(x)
    x = Dense(hidden_nodes, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(hidden_nodes, activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(2, activation='linear')(x)
    model = Model(inputs=[features_in, id_in], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(optimizer=opt, loss=crps_cost_function, 
                  metrics=[crps_cost_function])
    return model

In [262]:
emb_model = build_embedding_model2(features_train.shape[1], 537, 10, 100)

/Users/stephanrasp/anaconda/envs/py36_keras/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/Users/stephanrasp/anaconda/envs/py36_keras/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [263]:
emb_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_36 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_10 (Embedding)         (None, 1, 10)         5380        input_36[0][0]                   
____________________________________________________________________________________________________
input_35 (InputLayer)            (None, 24)            0                                            
____________________________________________________________________________________________________
flatten_8 (Flatten)              (None, 10)            0           embedding_10[0][0]               
___________________________________________________________________________________________

In [265]:
emb_model.fit([features_train, id_array_train], target_train, epochs=10, batch_size=1024,
                 validation_data=[[features_test, id_array_test], target_test], verbose=1)

Train on 180849 samples, validate on 181718 samples
Epoch 1/10
180849/180849 [==============================] - 4s - loss: 1.4796 - crps_cost_function: 1.4796 - val_loss: 1.9635 - val_crps_cost_function: 1.9635
Epoch 2/10
180849/180849 [==============================] - 4s - loss: 1.4621 - crps_cost_function: 1.4621 - val_loss: 1.8854 - val_crps_cost_function: 1.8854
Epoch 3/10
180849/180849 [==============================] - 4s - loss: 1.4490 - crps_cost_function: 1.4490 - val_loss: 1.9152 - val_crps_cost_function: 1.9152
Epoch 4/10
180849/180849 [==============================] - 4s - loss: 1.4429 - crps_cost_function: 1.4429 - val_loss: 1.8957 - val_crps_cost_function: 1.8957
Epoch 5/10
180849/180849 [==============================] - 4s - loss: 1.4252 - crps_cost_function: 1.4252 - val_loss: 1.7479 - val_crps_cost_function: 1.7479
Epoch 6/10
180849/180849 [==============================] - 4s - loss: 1.4195 - crps_cost_function: 1.4195 - val_loss: 1.8368 - val_crps_cost_function: 1